In [28]:
import numpy as np
import statsmodels.api as sm

In [29]:
def numpy_describe(data):
    return {
        'count': data.size,
        'mean': np.mean(data),
        'std': np.std(data),
        'min': np.min(data),
        '25%': np.percentile(data, 25),
        '50%': np.median(data),
        '75%': np.percentile(data, 75),
        'max': np.max(data)
    }

In [30]:
# set seed and number of observations

np.random.seed(1234)
n = 10000

In [31]:
# generate potential outcomes y0 and y1

y0 = np.random.normal(loc=5, scale=2, size=n)     # loc gives the mean (center), scale gives the standard deviation (spread or width) and size gives the output shape
y1 = np.random.normal(loc=3, scale=3, size=n)


In [32]:
numpy_describe(y0)

{'count': 10000,
 'mean': 5.032252920094396,
 'std': 1.9903598111878429,
 'min': -2.761796822673216,
 '25%': 3.7059922342911786,
 '50%': 5.036171702952106,
 '75%': 6.3673222747998555,
 'max': 11.575575794344036}

In [33]:
numpy_describe(y1)

{'count': 10000,
 'mean': 2.993513067511328,
 'std': 2.982285146387646,
 'min': -8.453567599471421,
 '25%': 0.9713242669213928,
 '50%': 2.9688533869384752,
 '75%': 5.007346796864675,
 'max': 14.104887297872574}

In [34]:
# generate delta

delta = y1 - y0

numpy_describe(delta)

{'count': 10000,
 'mean': -2.0387398525830682,
 'std': 3.596019942690159,
 'min': -17.78560429177407,
 '25%': -4.483421007554596,
 '50%': -2.0298129794790816,
 '75%': 0.36692827783590365,
 'max': 11.264692643618698}

In [39]:
# ATE - average treatment effect is the population average of all individual treatment effects
# E[delta] = E[Y1 - Y0] = E[Y1] - E[Y0]

ate = np.mean(delta)

print(f"ATE: {ate}")

ATE: -2.0387398525830682


In [36]:
# If the selection of the treatment is made on y0
# If a person's y0 is above 6.3 (above 75th percentile, they get treated

d = (y0 >= 6.3).astype(int)

# here we are basing the treatment assignment on y0, D is the treatment assignment which selects one of the potential outcomes based on the switching equation
# Yi = Di * Y1i + (1 - Di) * Y0i
# which means the individual i's realized outcomes Yi is determined by the treatment assignment Di which selects one of the potential outcomes
# Yi = Y1i if Di = 1
# Yi = Y0i if Di = 0

In [40]:
# ATT - Average treatment effect on the treated
# E[delta|D = 1] = E[Y1 - Y0|D = 1] = E[Y1|D = 1] - E[Y0|D = 1]

att = np.mean(delta[d == 1])

print(f"ATT: {att}")

ATT: -4.524642570784161


In [41]:
# ATU - average treatment effect on the untreated
# E[delta|D = 0] = E[Y1 - Y0|D = 0] = E[Y1|D = 0] - E[Y0|D = 0]

atu = np.mean(delta[d == 0])

print(f"ATU: {atu}")

ATU: -1.1520978438799516


In [42]:
# E[Y0|D = 1]

ey01 = np.mean(y0[d == 1])

print(f"E[Y0|D=1]: {ey01}")

E[Y0|D=1]: 7.499934994080874


In [43]:
# E[Y0|D = 0]

ey00 = np.mean(y0[d == 0])

print(f"E[Y0|D=0]: {ey00}")

E[Y0|D=0]: 4.15210963254719


In [44]:
# Selection bias
# selection bias = E[Y0|D=1] - E[Y0|D=0]

selection_bias = ey01 - ey00

print(f"Selection bias: {selection_bias}")

Selection bias: 3.347825361533684


In [45]:
# Proportion treated
# this term gives us the pi (part of the population that is treated) needed for the heterogeneous treatment effect bias
# heterogeneous treatment effect bias = (1 - pi)(ATT - ATU)

pi = np.mean(d)

print(f"Pi (Proportion treated): {pi}")

Pi (Proportion treated): 0.2629


In [47]:
hetero_treatment_effect_bias = (1 - pi) * (att - atu)

print(f"Heterogeneous treatment effect bias: {hetero_treatment_effect_bias}")

Heterogeneous treatment effect bias: -2.485902718201093


In [46]:
# Observed outcome (realized outcome) based on the switching equation mentioned above

y = d * y1 + (1 - d) * y0

print(f"Observed outcome: {y}")

Observed outcome: [5.94287033 2.61804861 4.99872323 ... 2.89507861 4.0046138  4.4879876 ]


In [48]:
# Simple difference in mean outcomes (SDO)
# SDO is essentially E[Y1|D=1] - E[Y0|D=0]
# when we estimate it, it becomes a sum of three things ATE + selection bias + heterogeneous treatment effect bias

sdo = ate + selection_bias + hetero_treatment_effect_bias

print(f"SDO: {sdo}")

SDO: -1.1768172092504772


In [52]:
# Total bias is SDO - ATE in the estimation

bias = sdo - ate

print(f"Bias: {bias}")

Bias: 0.861922643332591


In [49]:
# Let us see what regression of y (realized/observed outcome) on d (treatment assignment) is

X = sm.add_constant(d)
model_select_on_y0 = sm.OLS(y, X).fit()
print(model_select_on_y0.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     688.7
Date:                Sun, 06 Oct 2024   Prob (F-statistic):          7.48e-147
Time:                        15:32:19   Log-Likelihood:                -20989.
No. Observations:               10000   AIC:                         4.198e+04
Df Residuals:                    9998   BIC:                         4.200e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1521      0.023    180.584      0.0

In [50]:
# the E[Y0|D=0] is the coefficient for the constant
# the SDO is the coefficient of the x1

In [51]:
# if the selection of the treatment is on y1
# if a person's y1 is above 1 (above 25th percentile), they get treated

d = (y1 >= 1).astype(int)

# treatment assignment d is based on y1

In [54]:
# Repeating the calcs for this treatment

# ATT
att = np.mean(delta[d == 1])
print(f"ATT: {att}")

# ATU
atu = np.mean(delta[d == 0])
print(f"ATU: {atu}")

# E[Y0|D=1]
ey01 = np.mean(y0[d == 1])
print(f"E[Y0|D=1]: {ey01}")

# E[Y0|D=0]
ey00 = np.mean(y0[d == 0])
print(f"E[Y0|D=0]: {ey00}")

# Selection Bias
selection_bias = ey01 - ey00
print(f"Selection Bias: {selection_bias}")

# Proportion treated
pi = np.mean(d)
print(f"Pi (Proportion treated): {pi}")

# Heterogeneous treatment effect bias
hetero_treatment_effect_bias = (1 - pi) * (att - atu)
print(f"Heterogeneous treatment effect bias: {hetero_treatment_effect_bias}")

# Observed outcome y
y = d * y1 + (1 - d) * y0  # Switching equation

# Simple Difference in mean Outcomes (sdo)
sdo = ate + selection_bias + hetero_treatment_effect_bias
print(f"SDO: {sdo}")

# Regression of y on d
X = sm.add_constant(d)
model_select_on_y1 = sm.OLS(y, X).fit()
print(model_select_on_y1.summary())

# Bias
bias = sdo - ate
print(f"Bias: {bias}")

ATT: -0.7646986198916363
ATU: -5.790502299147778
E[Y0|D=1]: 5.037594206309802
E[Y0|D=0]: 5.016524043724377
Selection Bias: 0.021070162585424157
Pi (Proportion treated): 0.7465
Heterogeneous treatment effect bias: 1.2740412326914314
SDO: -0.7436284573062126
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     232.3
Date:                Sun, 06 Oct 2024   Prob (F-statistic):           7.29e-52
Time:                        15:44:14   Log-Likelihood:                -21715.
No. Observations:               10000   AIC:                         4.343e+04
Df Residuals:                    9998   BIC:                         4.345e+04
Df Model:                           1                                         
Covariance Type:            nonr

In [55]:
# ATE stays the same
# ATTs and ATUs change as it is different set of people in each treatment

In [ ]:
# 